In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pandas import DataFrame

In [3]:
url = "https://www.camara.leg.br/deputados/74646"
urlOrçamento = "https://www.camara.leg.br/cota-parlamentar/sumarizado?nuDeputadoId=233&dataInicio=1/2020&dataFim=1/2020&despesa=&nomeHospede=&nomePassageiro=&nomeFornecedor=&cnpjFornecedor=&numDocumento=&sguf=&filtroNivel1=1&filtroNivel2=2&filtroNivel3=3"

In [5]:
#PEGANDO DADOS DO NOME

option = Options()
option.headless = True
driver = webdriver.Chrome("C:\WebDriver/chromedriver.exe")
driver.get(url)
driver.find_element_by_xpath(
    "//div[@class='mm-page mm-slideout']//div[@class='js-mmenu-container']//div[@class='container-blocos-deputado']")
element = driver.find_element_by_xpath(
    "//div[@class='mm-page mm-slideout']//div[@class='js-mmenu-container']//div[@class='container-blocos-deputado']")

html_content = element.get_attribute("outerHTML")

soup = BeautifulSoup(html_content,'html.parser')

mylist = soup.find_all('h2', class_='nome-deputado')

nome=[]
for x in mylist:
    nome.append(str(x))
dados = []
mylist = soup.find_all('li')
for item in mylist:
    dados.append(item.text)
    
#PEGANDO DADOS DO ORÇAMENTO

driver.get(urlOrçamento)
driver.find_element_by_xpath(
    "//div[@class='coluna-centro col-md-9']")
elementOrçamento = driver.find_element_by_xpath(
    "//div[@class='coluna-centro col-md-9']")

html_contentOrçamento = elementOrçamento.get_attribute("outerHTML")

soupOrçamento = BeautifulSoup(html_contentOrçamento,'html.parser')





    
    
#TRATANDO DADOS DO NOME    

dadosjuntos = [[dados[0].split(":",1)[1],
                dados[1].split(":",1)[1],
                dados[2].split(":",1)[1],
                " ".join(dados[3].split(":",1)[1].replace('\n', '').split()),
                " ".join(dados[4].split(":",1)[1].replace('\n', '').split()),
                " ".join(dados[5].split(":",1)[1].replace('\n', '').split())]]

#TRATANDO DADOS DO ORÇAMENTO
Contagem = 0
tabeladados = []

while True:
    try:
        TipoDespesa = soupOrçamento.find(id="linkSumarizado{}".format(Contagem))
        quotes = [quote for quote in TipoDespesa]
        Despesa = soupOrçamento.find(id="sumarizado{}".format(Contagem))
        quotes2 = [quote for quote in Despesa]
        tabeladados.append([quotes[0].replace('\n', '').replace('\t', ''),quotes2[0].replace('\n', '').replace('\t', '')])
        
        Contagem += 1
        
    except:
        break

mes = soupOrçamento.find_all("h3",class_= "header")

dadomes =[]
listmes = [quote for quote in mes]

for x in listmes:
    dadomes.append(str(listmes[1]))
        
        
        
        
#JUNTANDO OS DADOS
tabelafinal = []
for x in range(len(tabeladados)):
    tabelafinal.append([dadosjuntos[0][0],dadosjuntos[0][1],dadosjuntos[0][2],dadosjuntos[0][3],
                        dadosjuntos[0][4],dadosjuntos[0][5],dadomes[0][28:-5],tabeladados[x][0],tabeladados[x][1]])
    
df = DataFrame(tabelafinal,columns=["Nome civil",'email','Telefone',
                                    'Endereço','Data de Nascimento','Naturalidade',"Mês","Tipo Despesa","Despesa"])

df

,Nome civil,email,Telefone,Endereço,Data de Nascimento,Naturalidade,Mês,Tipo Despesa,Despesa
0,AÉCIO NEVES DA CUNHA,dep.aecioneves@camara.leg.br,(61) 3215-5964,Gabinete 20 - Ed. principal - Câmara dos Deput...,10/03/1960,Belo Horizonte - MG,Janeiro/2020,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,"R$ 9.203,18"
1,AÉCIO NEVES DA CUNHA,dep.aecioneves@camara.leg.br,(61) 3215-5964,Gabinete 20 - Ed. principal - Câmara dos Deput...,10/03/1960,Belo Horizonte - MG,Janeiro/2020,TELEFONIA,"R$ 85,49"
2,AÉCIO NEVES DA CUNHA,dep.aecioneves@camara.leg.br,(61) 3215-5964,Gabinete 20 - Ed. principal - Câmara dos Deput...,10/03/1960,Belo Horizonte - MG,Janeiro/2020,SERVIÇOS POSTAIS,"R$ 1.644,88"
3,AÉCIO NEVES DA CUNHA,dep.aecioneves@camara.leg.br,(61) 3215-5964,Gabinete 20 - Ed. principal - Câmara dos Deput...,10/03/1960,Belo Horizonte - MG,Janeiro/2020,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.","R$ 2.000,00"
4,AÉCIO NEVES DA CUNHA,dep.aecioneves@camara.leg.br,(61) 3215-5964,Gabinete 20 - Ed. principal - Câmara dos Deput...,10/03/1960,Belo Horizonte - MG,Janeiro/2020,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,"R$ 6.273,01"
5,AÉCIO NEVES DA CUNHA,dep.aecioneves@camara.leg.br,(61) 3215-5964,Gabinete 20 - Ed. principal - Câmara dos Deput...,10/03/1960,Belo Horizonte - MG,Janeiro/2020,PASSAGEM AÉREA - REEMBOLSO,"R$ 2.188,68"
